# Homework and bake-off: word-level entailment with neural networks

In [1]:
__author__ = "Christopher Potts"
__version__ = "CS224u, Stanford, Spring 2020"

## Contents

1. [Overview](#Overview)
1. [Set-up](#Set-up)
1. [Data](#Data)
  1. [Edge disjoint](#Edge-disjoint)
  1. [Word disjoint](#Word-disjoint)
1. [Baseline](#Baseline)
  1. [Representing words: vector_func](#Representing-words:-vector_func)
  1. [Combining words into inputs: vector_combo_func](#Combining-words-into-inputs:-vector_combo_func)
  1. [Classifier model](#Classifier-model)
  1. [Baseline results](#Baseline-results)
1. [Homework questions](#Homework-questions)
  1. [Hypothesis-only baseline [2 points]](#Hypothesis-only-baseline-[2-points])
  1. [Alternatives to concatenation [2 points]](#Alternatives-to-concatenation-[2-points])
  1. [A deeper network [2 points]](#A-deeper-network-[2-points])
  1. [Your original system [3 points]](#Your-original-system-[3-points])
1. [Bake-off [1 point]](#Bake-off-[1-point])

## Overview

The general problem is word-level natural language inference.

Training examples are pairs of words $(w_{L}, w_{R}), y$ with $y = 1$ if $w_{L}$ entails $w_{R}$, otherwise $0$.

The homework questions below ask you to define baseline models for this and develop your own system for entry in the bake-off, which will take place on a held-out test-set distributed at the start of the bake-off. (Thus, all the data you have available for development is available for training your final system before the bake-off begins.)

<img src="fig/wordentail-diagram.png" width=600 alt="wordentail-diagram.png" />

## Set-up

See [the first notebook in this unit](nli_01_task_and_data.ipynb) for set-up instructions.

In [2]:
from collections import defaultdict
import json
import numpy as np
import os
import pandas as pd
from torch_shallow_neural_classifier import TorchShallowNeuralClassifier
import nli
import utils

In [3]:
DATA_HOME = 'data'

NLIDATA_HOME = os.path.join(DATA_HOME, 'nlidata')

wordentail_filename = os.path.join(
    NLIDATA_HOME, 'nli_wordentail_bakeoff_data.json')

GLOVE_HOME = os.path.join(DATA_HOME, 'glove.6B')

## Data

I've processed the data into two different train/test splits, in an effort to put some pressure on our models to actually learn these semantic relations, as opposed to exploiting regularities in the sample.

* `edge_disjoint`: The `train` and `dev` __edge__ sets are disjoint, but many __words__ appear in both `train` and `dev`.
* `word_disjoint`: The `train` and `dev` __vocabularies are disjoint__, and thus the edges are disjoint as well.

These are very different problems. For `word_disjoint`, there is real pressure on the model to learn abstract relationships, as opposed to memorizing properties of individual words.

In [4]:
with open(wordentail_filename) as f:
    wordentail_data = json.load(f)

The outer keys are the  splits plus a list giving the vocabulary for the entire dataset:

In [5]:
wordentail_data.keys()

dict_keys(['edge_disjoint', 'vocab', 'word_disjoint'])

### Edge disjoint

In [6]:
wordentail_data['edge_disjoint'].keys()

dict_keys(['dev', 'train'])

This is what the split looks like; all three have this same format:

In [7]:
wordentail_data['edge_disjoint']['dev'][: 5]

[[['sweater', 'stroke'], 0],
 [['constipation', 'hypovolemia'], 0],
 [['disease', 'inflammation'], 0],
 [['herring', 'animal'], 1],
 [['cauliflower', 'outlook'], 0]]

Let's test to make sure no edges are shared between `train` and `dev`:

In [8]:
nli.get_edge_overlap_size(wordentail_data, 'edge_disjoint')

0

As we expect, a *lot* of vocabulary items are shared between `train` and `dev`:

In [9]:
nli.get_vocab_overlap_size(wordentail_data, 'edge_disjoint')

2916

This is a large percentage of the entire vocab:

In [10]:
len(wordentail_data['vocab'])

8470

Here's the distribution of labels in the `train` set. It's highly imbalanced, which will pose a challenge for learning. (I'll go ahead and reveal that the `dev` set is similarly distributed.)

In [11]:
def label_distribution(split):
    return pd.DataFrame(wordentail_data[split]['train'])[1].value_counts()

In [12]:
label_distribution('edge_disjoint')

0    14650
1     2745
Name: 1, dtype: int64

### Word disjoint

In [13]:
wordentail_data['word_disjoint'].keys()

dict_keys(['dev', 'train'])

In the `word_disjoint` split, no __words__ are shared between `train` and `dev`:

In [14]:
nli.get_vocab_overlap_size(wordentail_data, 'word_disjoint')

0

Because no words are shared between `train` and `dev`, no edges are either:

In [15]:
nli.get_edge_overlap_size(wordentail_data, 'word_disjoint')

0

The label distribution is similar to that of `edge_disjoint`, though the overall number of examples is a bit smaller:

In [16]:
label_distribution('word_disjoint')

0    7199
1    1349
Name: 1, dtype: int64

## Baseline

Even in deep learning, __feature representation is vital and requires care!__ For our task, feature representation has two parts: representing the individual words and combining those representations into a single network input.

### Representing words: vector_func

Let's consider two baseline word representations methods:

1. Random vectors (as returned by `utils.randvec`).
1. 50-dimensional GloVe representations.

In [17]:
def randvec(w, n=50, lower=-1.0, upper=1.0):
    """Returns a random vector of length `n`. `w` is ignored."""
    return utils.randvec(n=n, lower=lower, upper=upper)

In [18]:
# Any of the files in glove.6B will work here:

glove_dim = 50

glove_src = os.path.join(GLOVE_HOME, 'glove.6B.{}d.txt'.format(glove_dim))

# Creates a dict mapping strings (words) to GloVe vectors:
GLOVE = utils.glove2dict(glove_src)

def glove_vec(w):    
    """Return `w`'s GloVe representation if available, else return 
    a random vector."""
    return GLOVE.get(w, randvec(w, n=glove_dim))

### Combining words into inputs: vector_combo_func

Here we decide how to combine the two word vectors into a single representation. In more detail, where `u` is a vector representation of the left word and `v` is a vector representation of the right word, we need a function `vector_combo_func` such that `vector_combo_func(u, v)` returns a new input vector `z` of dimension `m`. A simple example is concatenation:

In [19]:
def vec_concatenate(u, v):
    """Concatenate np.array instances `u` and `v` into a new np.array"""
    return np.concatenate((u, v))

`vector_combo_func` could instead be vector average, vector difference, etc. (even combinations of those) – there's lots of space for experimentation here; [homework question 2](#Alternatives-to-concatenation-[1-point]) below pushes you to do some exploration.

### Classifier model

For a baseline model, I chose `TorchShallowNeuralClassifier`:

In [20]:
net = TorchShallowNeuralClassifier(hidden_dim=50, max_iter=100)

### Baseline results

The following puts the above pieces together, using `vector_func=glove_vec`, since `vector_func=randvec` seems so hopelessly misguided for `word_disjoint`!

In [21]:
word_disjoint_experiment = nli.wordentail_experiment(
    train_data=wordentail_data['word_disjoint']['train'],
    assess_data=wordentail_data['word_disjoint']['dev'], 
    model=net, 
    vector_func=glove_vec,
    vector_combo_func=vec_concatenate)

Finished epoch 100 of 100; error is 0.020413828897289932              precision    recall  f1-score   support

           0      0.925     0.936     0.931      1910
           1      0.438     0.397     0.417       239

    accuracy                          0.876      2149
   macro avg      0.682     0.667     0.674      2149
weighted avg      0.871     0.876     0.874      2149



## Homework questions

Please embed your homework responses in this notebook, and do not delete any cells from the notebook. (You are free to add as many cells as you like as part of your responses.)

### Hypothesis-only baseline [2 points]

During our discussion of SNLI and MultiNLI, we noted that a number of research teams have shown that hypothesis-only baselines for NLI tasks can be remarkably robust. This question asks you to explore briefly how this baseline effects the 'edge_disjoint' and 'word_disjoint' versions of our task.

For this problem, submit two functions:

1. A `vector_combo_func` function called `hypothesis_only` that simply throws away the premise, using the unmodified hypothesis (second) vector as its representation of the example.

1. A function called `run_hypothesis_only_evaluation` that does the following:
    1. Loops over the two conditions 'word_disjoint' and 'edge_disjoint' and the two `vector_combo_func` values `vec_concatenate` and `hypothesis_only`, calling `nli.wordentail_experiment` to train on the conditions 'train' portion and assess on its 'dev' portion, with `glove_vec` as the `vector_func`. So that the results are consistent, use an `sklearn.linear_model.LogisticRegression` with default parameters as the model.
    1. Returns a `dict` mapping `(condition_name, function_name)` pairs to the 'macro-F1' score for that pair, as returned by the call to `nli.wordentail_experiment`. (Tip: you can get the `str` name of your function `hypothesis_only` with `hypothesis_only.__name__`.)
    
The test functions `test_hypothesis_only` and `test_run_hypothesis_only_evaluation` will help ensure that your functions have the desired logic.

In [66]:
##### YOUR CODE HERE
from sklearn.linear_model import LogisticRegression

def hypothesis_only(u, v):
    return v



def run_hypothesis_only_evaluation(include_shallow=False):
    ##### YOUR CODE HERE
    results = {}
    model_facts = [lambda: LogisticRegression()]
    if include_shallow:
        model_facts.append(lambda: TorchShallowNeuralClassifier(hidden_dim=50, max_iter=100))
    for condition in [ 'word_disjoint', 'edge_disjoint']:
        for vector_combo_func in [vec_concatenate, hypothesis_only]:
            for model_fact in model_facts:
                model = model_fact()
                iter_name = (condition, vector_combo_func.__name__) 
                if include_shallow:
                    iter_name = iter_name + (model.__class__.__name__,)
                print("Running {}".format(iter_name))
                exp = nli.wordentail_experiment(
                    train_data=wordentail_data[condition]['train'],
                    assess_data=wordentail_data[condition]['dev'], 
                    model=model, 
                    vector_func=glove_vec,
                    vector_combo_func=vector_combo_func
                )   
                results[iter_name] = exp['macro-F1']
    pres = []
    for itr in results: 
        pres.append(("{:<15}".format(str(itr)+':'), results[itr]))
    pres = sorted(pres, key=lambda x:x[1])
    for r in pres: print(r[0], r[1])
    return results



In [67]:
def test_hypothesis_only(hypothesis_only):
    v = hypothesis_only(1, 2)
    assert v == 2   

In [68]:
test_hypothesis_only(hypothesis_only)

In [69]:
def test_run_hypothesis_only_evaluation(run_hypothesis_only_evaluation):
    results = run_hypothesis_only_evaluation()
    assert ('word_disjoint', 'vec_concatenate') in results, \
        "The return value of `run_hypothesis_only_evaluation` does not have the intended kind of keys"
    assert isinstance(results[('word_disjoint', 'vec_concatenate')], float), \
        "The values of the `run_hypothesis_only_evaluation` result should be floats"

In [70]:
test_run_hypothesis_only_evaluation(run_hypothesis_only_evaluation)

Running ('word_disjoint', 'vec_concatenate')
              precision    recall  f1-score   support

           0      0.902     0.979     0.939      1910
           1      0.467     0.146     0.223       239

    accuracy                          0.886      2149
   macro avg      0.684     0.563     0.581      2149
weighted avg      0.853     0.886     0.859      2149

Running ('word_disjoint', 'hypothesis_only')
              precision    recall  f1-score   support

           0      0.893     0.989     0.939      1910
           1      0.382     0.054     0.095       239

    accuracy                          0.885      2149
   macro avg      0.638     0.522     0.517      2149
weighted avg      0.836     0.885     0.845      2149

Running ('edge_disjoint', 'vec_concatenate')
              precision    recall  f1-score   support

           0      0.876     0.970     0.921      7376
           1      0.583     0.232     0.332      1321

    accuracy                          0.858    

In [72]:
#run_hypothesis_only_evaluation(include_shallow=True)

Running ('word_disjoint', 'vec_concatenate', 'LogisticRegression')
              precision    recall  f1-score   support

           0      0.902     0.978     0.938      1910
           1      0.455     0.146     0.222       239

    accuracy                          0.886      2149
   macro avg      0.678     0.562     0.580      2149
weighted avg      0.852     0.886     0.859      2149

Running ('word_disjoint', 'vec_concatenate', 'TorchShallowNeuralClassifier')
Finished epoch 100 of 100; error is 0.023769455030560493              precision    recall  f1-score   support

           0      0.923     0.936     0.930      1910
           1      0.425     0.377     0.399       239

    accuracy                          0.874      2149
   macro avg      0.674     0.656     0.664      2149
weighted avg      0.868     0.874     0.871      2149

Running ('word_disjoint', 'hypothesis_only', 'LogisticRegression')
              precision    recall  f1-score   support

           0      0.893 

{('word_disjoint',
  'vec_concatenate',
  'LogisticRegression'): 0.5798704931686259,
 ('word_disjoint',
  'vec_concatenate',
  'TorchShallowNeuralClassifier'): 0.6643342899151987,
 ('word_disjoint',
  'hypothesis_only',
  'LogisticRegression'): 0.5166301461305162,
 ('word_disjoint',
  'hypothesis_only',
  'TorchShallowNeuralClassifier'): 0.533358234451438,
 ('edge_disjoint', 'vec_concatenate', 'LogisticRegression'): 0.624037525556164,
 ('edge_disjoint',
  'vec_concatenate',
  'TorchShallowNeuralClassifier'): 0.7435023725559298,
 ('edge_disjoint',
  'hypothesis_only',
  'LogisticRegression'): 0.6062178665830953,
 ('edge_disjoint',
  'hypothesis_only',
  'TorchShallowNeuralClassifier'): 0.7284395181500279}

### Alternatives to concatenation [2 points]

We've so far just used vector concatenation to represent the premise and hypothesis words. This question asks you to explore two simple alternative:

1. Write a function `vec_diff` that, for a given pair of vector inputs `u` and `v`, returns the element-wise difference between `u` and `v`.

1. Write a function `vec_max` that, for a given pair of vector inputs `u` and `v`, returns the element-wise max values between `u` and `v`.

You needn't include your uses of `nli.wordentail_experiment` with these functions, but we assume you'll be curious to see how they do!

In [73]:
def vec_diff(u, v):
    return u-v



    
def vec_max(u, v):
    return np.maximum(u,v)




In [74]:
def test_vec_diff(vec_diff):
    u = np.array([10.2, 8.1])
    v = np.array([1.2, -7.1])
    result = vec_diff(u, v)
    expected = np.array([9.0, 15.2])
    assert np.array_equal(result, expected), \
        "Expected {}; got {}".format(expected, result)

In [75]:
test_vec_diff(vec_diff)

In [76]:
def test_vec_max(vec_max):
    u = np.array([1.2,  8.1])
    v = np.array([10.2, -7.1])
    result = vec_max(u, v)
    expected = np.array([10.2, 8.1])
    assert np.array_equal(result, expected), \
        "Expected {}; got {}".format(expected, result)

In [77]:
test_vec_max(vec_max)

### A deeper network [2 points]

It is very easy to subclass `TorchShallowNeuralClassifier` if all you want to do is change the network graph: all you have to do is write a new `define_graph`. If your graph has new arguments that the user might want to set, then you should also redefine `__init__` so that these values are accepted and set as attributes.

For this question, please subclass `TorchShallowNeuralClassifier` so that it defines the following graph:

$$\begin{align}
h_{1} &= xW_{1} + b_{1} \\
r_{1} &= \textbf{Bernoulli}(1 - \textbf{dropout\_prob}, n) \\
d_{1} &= r_1 * h_{1} \\
h_{2} &= f(d_{1}) \\
h_{3} &= h_{2}W_{2} + b_{2}
\end{align}$$

Here, $r_{1}$ and $d_{1}$ define a dropout layer: $r_{1}$ is a random binary vector of dimension $n$, where the probability of a value being $1$ is given by $1 - \textbf{dropout_prob}$. $r_{1}$ is multiplied element-wise by our first hidden representation, thereby zeroing out some of the values. The result is fed to the user's activation function $f$, and the result of that is fed through another linear layer to produce $h_{3}$. (Inside `TorchShallowNeuralClassifier`, $h_{3}$ is the basis for a softmax classifier, so no activation function is applied to it.)

For your implementation, please use `nn.Sequential`, `nn.Linear`, and `nn.Dropout` to define the required layers.

For comparison, using this notation, `TorchShallowNeuralClassifier` defines the following graph:

$$\begin{align}
h_{1} &= xW_{1} + b_{1} \\
h_{2} &= f(h_{1}) \\
h_{3} &= h_{2}W_{2} + b_{2}
\end{align}$$

The following code starts this sub-class for you, so that you can concentrate on `define_graph`. Be sure to make use of `self.dropout_prob`

For this problem, submit just your completed  `TorchDeepNeuralClassifier`. You needn't evaluate it, though we assume you will be keen to do that!

You can use `test_TorchDeepNeuralClassifier` to ensure that your network has the intended structure.

In [78]:
import torch.nn as nn

class TorchDeepNeuralClassifier(TorchShallowNeuralClassifier):
    def __init__(self, dropout_prob=0.7, **kwargs):
        self.dropout_prob = dropout_prob
        super().__init__(**kwargs)
    
    def define_graph(self):
        """Complete this method!
        
        Returns
        -------
        an `nn.Module` instance, which can be a free-standing class you 
        write yourself, as in `torch_rnn_classifier`, or the outpiut of 
        `nn.Sequential`, as in `torch_shallow_neural_classifier`.
        
        """
        return nn.Sequential(
            nn.Linear(self.input_dim, self.hidden_dim),
            nn.Dropout(self.dropout_prob),
            self.hidden_activation,
            nn.Linear(self.hidden_dim, self.n_classes_)
        )


    

##### YOUR CODE HERE    




In [79]:
def test_TorchDeepNeuralClassifier(TorchDeepNeuralClassifier):
    dropout_prob = 0.55
    assert hasattr(TorchDeepNeuralClassifier(), "dropout_prob"), \
        "TorchDeepNeuralClassifier must have an attribute `dropout_prob`."
    try:
        inst = TorchDeepNeuralClassifier(dropout_prob=dropout_prob)
    except TypeError:
        raise TypeError("TorchDeepNeuralClassifier must allow the user "
                        "to set `dropout_prob` on initialization")
    inst.input_dim = 10
    inst.n_classes_ = 5
    graph = inst.define_graph()
    assert len(graph) == 4, \
        "The graph should have 4 layers; yours has {}".format(len(graph))    
    expected = {
        0: 'Linear',
        1: 'Dropout',
        2: 'Tanh',
        3: 'Linear'}
    for i, label in expected.items():
        name = graph[i].__class__.__name__
        assert label in name, \
            "The {} layer of the graph should be a {} layer; yours is {}".format(i, label, name)
    assert graph[1].p == dropout_prob, \
        "The user's value for `dropout_prob` should be the value of `p` for the Dropout layer."

In [80]:
test_TorchDeepNeuralClassifier(TorchDeepNeuralClassifier)

### Your original system [3 points]

This is a simple dataset, but our focus on the 'word_disjoint' condition ensures that it's a challenging one, and there are lots of modeling strategies one might adopt. 

You are free to do whatever you like. We require only that your system differ in some way from those defined in the preceding questions. They don't have to be completely different, though. For example, you might want to stick with the model but represent examples differently, or the reverse.

Keep in mind that, for the bake-off evaluation, the 'edge_disjoint' portions of the data are off limits. You can, though, train on the combination of the 'word_disjoint' 'train' and 'dev' portions. You are free to use different pretrained word vectors and the like. Please do not introduce additional entailment datasets into your training data, though.

Please embed your code in this notebook so that we can rerun it.

In the cell below, please provide a brief technical description of your original system, so that the teaching team can gain an understanding of what it does. This will help us to understand your code and analyze all the submissions to identify patterns and strategies.

In [81]:
# Enter your system description in this cell.
# Please do not remove this comment.
"""
For this particular task two major directions have been identified: that of employing a MLP on a particular 
representation of the input (u,v) tuple or that of creating architectures based on siamese networks. Although the 
MLP approach seems more direct the main area that has been explored is that of simease-based architectures.

The overall approach was based on three steps:

(1) define the baselines - see `get_baselines` function - that have both the above mentioned simple MLP approach 
as well as logistic regression 

(2) prepare a overall model architecture `ThWordEntailModel` incapsulated in `WordEntailClassifier` model wrapper that 
would allow us to experiment with classic siamese networks approaches such as "Dimensionality Reduction by Learning 
an Invariant Mapping" or "Siamese Neural Networks for One-shot Image Recognition" as well as approaches based on a 
graphs architecture with two-column separate encoders and a final classification MLP backbone.

(3) prepare a grid search strategy that would cycle most of the major architectures proposed by (2) - this particular 
feat is provided by `run_grid_search` function

(4) run and refine the random grid search and add features and/or new approaches. Two separated grids have been defined
anc concatenated - one for the non-contrastive-loss graphs and another for the ones based on contrastive loss

(5) select best architecture and re-run the experiment

Data preparation and training details:

The actual training procedure was based on early stopping combined with efficient learning rate decay (that includes
parameter/optimizer restoration). The early stopping mechanism used the proposed `dev_data`. Due to class imbalance and 
to the model tendency to favor "not entail", focal loss has been added in the experiment as a grid-search option vs standard
BCE. To further test the model capabilities 4 different datasets have been extracted from the original two datasets (train 
and dev): a train dataset where all words are contained by the  pretrained GloVe, a similar dev set and a train/dev couple
where either one or both words in each observation are not contained within the pretrained GloVe embeddings (see the
function `test_glove_vs_data`).

Due to the fact that both training and validation (as well as real-life) datasets have out-of-vocabulary words (out of the 
pretrained GloVe vocab) a "approaximator" function has been developed that generates word embeddings for the OOV words.
Below we can see a few examples of word-embeddings neighbors obtained based on the generated embedding (value is cosine
distance):

replacement for 'unwrought'
wrought        1.1327e-10
devastation    4.6164e-01
ironwork       4.7856e-01
wreaked        4.8097e-01
railings       5.1378e-01


replacement for 'haematemesis'
emesis                  1.3017e-10
diaphoresis             5.0699e-01
itraconazole            5.4648e-01
chemotherapy-induced    5.5422e-01
anovulation             5.5664e-01

replacement for 'pyrexia'
pyrex           9.7854e-11
corningware     4.8760e-01
borosilicate    5.2159e-01
bakeware        5.2440e-01
glassware       5.3898e-01

replacement for 'cervicitis'
cervi         1.0381e-10
severini      5.9782e-01
pasqualino    6.0177e-01
kalantar      6.0200e-01
conason       6.0281e-01

replacement for 'dacryocystitis'
cystitis          1.0738e-10
interstitial      4.8357e-01
pyelonephritis    5.2237e-01
endometriosis     5.3036e-01
bronchiolitis     5.4040e-01

replacement for 'antheridium'
anther      1.4220e-10
stamen      5.4327e-01
anthers     5.6385e-01
piasters    5.9172e-01
pistil      6.0421e-01


"""



## Bake-off [1 point]

The goal of the bake-off is to achieve the highest macro-average F1 score on __word_disjoint__, on a test set that we will make available at the start of the bake-off. The announcement will go out on the discussion forum. To enter, you'll be asked to run `nli.bake_off_evaluation` on the output of your chosen `nli.wordentail_experiment` run. 

The cells below this one constitute your bake-off entry.

The rules described in the [Your original system](#Your-original-system-[3-points]) homework question are also in effect for the bake-off.

Systems that enter will receive the additional homework point, and systems that achieve the top score will receive an additional 0.5 points. We will test the top-performing systems ourselves, and only systems for which we can reproduce the reported results will win the extra 0.5 points.

Late entries will be accepted, but they cannot earn the extra 0.5 points. Similarly, you cannot win the bake-off unless your homework is submitted on time.

The announcement will include the details on where to submit your entry.

In [ ]:
# Enter your bake-off assessment code into this cell. 
# Please do not remove this comment.
##### YOUR CODE HERE




In [ ]:
# On an otherwise blank line in this cell, please enter
# your macro-avg f1 value as reported by the code above. 
# Please enter only a number between 0 and 1 inclusive.
# Please do not remove this comment.

##### YOUR CODE HERE


